# Random checks

In [1]:
import os

import torch

In [5]:
from plant_leaves.model import PlantClassifier

model = PlantClassifier().to("mps:0")
model.load_state_dict(torch.load("../models/model.pth"))
model.eval()

onnx_model = torch.onnx.export(
    model,  # model being run
    (torch.randn(1, 3, 240, 240).to("mps"),),  # model input (or a tuple for multiple inputs)
    "plant_leaves_graph.onnx",  # where to save the model (can be a file or file-like object)
    input_names=["input"],  # the model's input names
    output_names=["output"],
)  # the model's output names

/var/folders/8h/82c89t014nv4j0sw212kn4640000gn/T/ipykernel_40479/2565670260.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("../models/m

In [25]:
import inspect

print(inspect.signature(model.forward))

(x: torch.Tensor) -> Any


In [15]:
pt = torch.load("../data/processed/test/targets.pt")

/var/folders/4f/7kcbgj992s5fhk7wcryyv4hc0000gn/T/ipykernel_11656/1032847024.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pt = torch.load('../data/processed/test/targe

In [16]:
pt.shape

torch.Size([15])

In [2]:
import os

path = "~"
os.path.dirname(os.path.dirname(os.path.abspath(path)))

'/Users/tzikos/Desktop/DTU/MLOps/Project/Plant_Leaves_Classification_MLOps_DTU02476'

In [4]:
import os

import torch

targets = torch.load("../data/processed/train/targets.pt")

/var/folders/4f/7kcbgj992s5fhk7wcryyv4hc0000gn/T/ipykernel_47195/1688312756.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  targets = torch.load("../data/processed/train

# Check pruning

In [20]:
import numpy as np
import onnx
import onnxruntime as ort

onnx_model = onnx.load("../models/model.onnx")


def prune_weights_randomly(onnx_model, prune_ratio=0.2):
    """
    Prunes weights in the ONNX model randomly.

    Parameters:
    - onnx_model: Loaded ONNX model.
    - prune_ratio: Fraction of weights to set to zero (0 to 1).

    Returns:
    - pruned_model: Modified ONNX model.
    """
    for initializer in onnx_model.graph.initializer:
        # Get weights as a numpy array
        weights = np.frombuffer(initializer.raw_data, dtype=np.float32)

        # Randomly zero out a fraction of weights
        mask = np.random.rand(*weights.shape) > prune_ratio
        pruned_weights = weights * mask

        # Update the initializer with pruned weights
        initializer.raw_data = pruned_weights.tobytes()

    return onnx_model


# Prune the model
pruned_model = prune_weights_randomly(onnx_model, prune_ratio=0.2)

# Save the pruned model
pruned_model_path = "pruned_model.onnx"
onnx.save(pruned_model, pruned_model_path)

In [21]:
import time

import numpy as np

# Load the pruned model
session = ort.InferenceSession(pruned_model_path)

# Prepare input data (example)
input_name = session.get_inputs()[0].name
input_shape = session.get_inputs()[0].shape
input_data = np.random.rand(*input_shape).astype(np.float32)

# Run inference
output_name = session.get_outputs()[0].name
outputs = session.run([output_name], {input_name: input_data})

print("Inference output:", outputs)

Inference output: [array([[0., 1.]], dtype=float32)]


In [22]:
from onnxruntime_tools import optimizer

optimized_model = optimizer.optimize_model(pruned_model_path)
optimized_model.save_model_to_file("optimized_model.onnx")

In [30]:
def measure_inference_time(model_path, input_shape, num_runs=100):
    """
    Measures the average inference time of an ONNX model.

    Parameters:
    - model_path: Path to the ONNX model.
    - input_shape: Shape of the input tensor.
    - num_runs: Number of inference runs to average.

    Returns:
    - avg_time: Average inference time in milliseconds.
    """
    session = ort.InferenceSession(model_path)
    input_name = session.get_inputs()[0].name
    input_data = np.random.rand(*input_shape).astype(np.float32)

    # Warm-up runs
    for _ in range(10000):
        session.run(None, {input_name: input_data})

    # Measure inference time
    start_time = time.time()
    for _ in range(num_runs):
        session.run(None, {input_name: input_data})
    end_time = time.time()

    avg_time = (end_time - start_time) / num_runs * 1000  # Convert to milliseconds
    return avg_time

In [31]:
# Get input shape from the original model
original_model_path = "../models/model.onnx"
optimized_pruned_model_path = "optimized_model.onnx"
session = ort.InferenceSession(original_model_path)
input_shape = session.get_inputs()[0].shape

# Measure inference times
original_time = measure_inference_time(original_model_path, input_shape)
pruned_time = measure_inference_time(pruned_model_path, input_shape)
optimized_pruned_time = measure_inference_time(optimized_pruned_model_path, input_shape)

# Display results
print(f"Original Model Inference Time: {original_time:.2f} ms")
print(f"Pruned Model Inference Time: {pruned_time:.2f} ms")
print(f"Optimized Pruned Model Inference Time: {optimized_pruned_time:.2f} ms")

Original Model Inference Time: 30.12 ms
Pruned Model Inference Time: 30.50 ms
Optimized Pruned Model Inference Time: 30.61 ms


In [11]:
import numpy as np


def compare_weights(model1_path, model2_path):
    """
    Compare weights between two ONNX models.

    Parameters:
    - model1_path: Path to the first ONNX model (original).
    - model2_path: Path to the second ONNX model (pruned).

    Returns:
    - weight_diff: A dictionary of weight differences.
    """
    model1 = onnx.load(model1_path)
    model2 = onnx.load(model2_path)

    weight_diff = {}
    for init1, init2 in zip(model1.graph.initializer, model2.graph.initializer):
        name = init1.name
        weights1 = np.frombuffer(init1.raw_data, dtype=np.float32)
        weights2 = np.frombuffer(init2.raw_data, dtype=np.float32)

        # Compute the difference
        diff = np.abs(weights1 - weights2).mean()
        weight_diff[name] = diff

    return weight_diff


# Compare weights
weight_differences = compare_weights("../models/model.onnx", "pruned_model.onnx")
print("Weight Differences:", weight_differences)

Weight Differences: {'backbone.blocks.0.0.se.conv_reduce.weight': np.float32(0.010197334), 'backbone.blocks.0.0.se.conv_reduce.bias': np.float32(0.27972263), 'backbone.blocks.0.0.se.conv_expand.weight': np.float32(0.009742614), 'backbone.blocks.0.0.se.conv_expand.bias': np.float32(0.31252247), 'backbone.blocks.0.1.se.conv_reduce.weight': np.float32(0.0097963065), 'backbone.blocks.0.1.se.conv_reduce.bias': np.float32(0.90575165), 'backbone.blocks.0.1.se.conv_expand.weight': np.float32(0.008433992), 'backbone.blocks.0.1.se.conv_expand.bias': np.float32(0.6111971), 'backbone.blocks.1.0.se.conv_reduce.weight': np.float32(0.0069128494), 'backbone.blocks.1.0.se.conv_reduce.bias': np.float32(2.034321), 'backbone.blocks.1.0.se.conv_expand.weight': np.float32(0.0066853017), 'backbone.blocks.1.0.se.conv_expand.bias': np.float32(0.41265675), 'backbone.blocks.1.1.se.conv_reduce.weight': np.float32(0.010195589), 'backbone.blocks.1.1.se.conv_reduce.bias': np.float32(0.054137204), 'backbone.blocks.1.

In [13]:
import pandas as pd

diff_df = pd.DataFrame(weight_differences.items(), columns=["Weight Name", "Difference"])
diff_df

,Weight Name,Difference
0,backbone.blocks.0.0.se.conv_reduce.weight,0.010197
1,backbone.blocks.0.0.se.conv_reduce.bias,0.279723
2,backbone.blocks.0.0.se.conv_expand.weight,0.009743
3,backbone.blocks.0.0.se.conv_expand.bias,0.312522
4,backbone.blocks.0.1.se.conv_reduce.weight,0.009796
...,...,...
227,onnx::Conv_1120,0.621893
228,onnx::Conv_1122,0.026424
229,onnx::Conv_1123,0.081530
230,onnx::Conv_1125,0.006889


In [16]:
def compare_model_structure(model1_path, model2_path):
    """
    Compare the structure of two ONNX models.

    Parameters:
    - model1_path: Path to the first ONNX model (original).
    - model2_path: Path to the second ONNX model (pruned).
    """
    model1 = onnx.load(model1_path)
    model2 = onnx.load(model2_path)

    # Count nodes and layers
    nodes1 = len(model1.graph.node)
    nodes2 = len(model2.graph.node)
    print(f"Original Model Nodes: {nodes1}")
    print(f"Pruned Model Nodes: {nodes2}")

    # Check node differences
    original_nodes = {node.name for node in model1.graph.node}
    pruned_nodes = {node.name for node in model2.graph.node}

    removed_nodes = original_nodes - pruned_nodes
    added_nodes = pruned_nodes - original_nodes

    print("Removed Nodes:", removed_nodes)
    print("Added Nodes:", added_nodes)


# Compare structure
compare_model_structure("../models/model.onnx", "pruned_model.onnx")

Original Model Nodes: 342
Pruned Model Nodes: 342
Removed Nodes: set()
Added Nodes: set()


In [19]:
def compare_model_outputs(model1_path, model2_path, input_data):
    """
    Compare the outputs of two ONNX models for the same input.

    Parameters:
    - model1_path: Path to the first ONNX model (original).
    - model2_path: Path to the second ONNX model (pruned).
    - input_data: Input data for inference.

    Returns:
    - output_diff: Mean absolute difference between the outputs.
    """
    session1 = ort.InferenceSession(model1_path)
    session2 = ort.InferenceSession(model2_path)

    input_name1 = session1.get_inputs()[0].name
    input_name2 = session2.get_inputs()[0].name

    output1 = session1.run(None, {input_name1: input_data})[0]
    output2 = session2.run(None, {input_name2: input_data})[0]

    # Compute difference
    diff = np.abs(output1 - output2).mean()
    return diff, output1, output2


# Prepare input data
input_shape = ort.InferenceSession("../models/model.onnx").get_inputs()[0].shape
input_data = np.random.rand(*input_shape).astype(np.float32)

# Compare outputs
output_difference, out1, out2 = compare_model_outputs("../models/model.onnx", "optimized_model.onnx", input_data)
print("Output Difference:", output_difference)
print("Output 1:", out1)
print("Output 2:", out2)

Output Difference: 0.36133397
Output 1: [[0.6386661 0.361334 ]]
Output 2: [[1. 0.]]
